# Исследование надежности заемщиков


## Откроем таблицу и изучим общую информацию о данных

**Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Выведем первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Выведем количество пропущенных значений для каждого столбца. Используем комбинацию двух методов.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
# циклом пройдемся по уникальным значениям столбца income_type {t = сотрудник, компаньон и т.д.}
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()
# проверим, сколько осталось отсутствующих значений в столбце total_income
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64


### Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()
# проверим что избавились от аномальных значений
print(data['days_employed'].unique())

[8437.67302776 4024.80375385 5623.42261023 ... 2113.3468877  3112.4817052
 1984.50758853]


**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец скорее всего не понадобится для исследования.

**Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедимся, что все пропуски заполнены. Выведем количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.**

In [14]:
data['education'] = data['education'].str.lower()

**Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

** На основании диапазонов, указанных ниже, создаем в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Пример: кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию `categorize_purpose()`, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
#print(data.head(20))

### Шаг 3. Исследуем данные и ответим на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
# количество детей - 'children' - от 0 до 5
# возврат кредита в срок - 'debt' - булева - 0 - всё в срок, 1 - есть невозврат кредита

########### решение через groupby ###############

# посчитаем среднюю долю невозврата по категории количество детей и выведем в процентах
data_1 = data.groupby('children')['debt'].mean()*100
print(data_1)
# а также выведем сумму невозвратов по количеству детей
data_2 = data.groupby('children')['debt'].sum()
print(data_2)

########### решение через pivot ###############

# создадим столбец с количеством невозвратов
data['credits'] = data['debt'].count() 
# создадим сводную таблицу, в которой в строках - количество детей, в столбце - количество невозвратов
pivot_debt = data.pivot_table(index=['children'], columns='credits', values='debt', aggfunc='sum')
#print(pivot_debt) 

# создадим сводную таблицу, в которой в строках - количество детей, в столбце - общее количество кредитов
pivot_credits = data.pivot_table(index=['children'], columns='credits', values='debt', aggfunc='count')
#print(pivot_credits)

# посчитаем среднюю долю невозврата
data_ratio = pivot_debt / pivot_credits * 100

#объединим количество невозвратов с общим количеством кредитов
result = pivot_debt.merge(pivot_credits, on='children')

#и присоединим к новой таблице среднюю долю невозврата
result = result.merge(data_ratio, on='children')
#print(result)
#result.sort_values(by='children', ascending= False)
#print(result)
result.rename(columns= {'21331_x':'Количество невозвратов','21331_y':'Количество кредитов',21331:'Процент задолжности, %'})

children
0    7.543822
1    9.234609
2    9.454191
3    8.181818
4    9.756098
5    0.000000
Name: debt, dtype: float64
children
0    1063
1     444
2     194
3      27
4       4
5       0
Name: debt, dtype: int64


credits,Количество невозвратов,Количество кредитов,"Процент задолжности, %"
children,,,
0,1063,14091,7.543822
1,444,4808,9.234609
2,194,2052,9.454191
3,27,330,8.181818
4,4,41,9.756098
5,0,9,0.000000


**Вывод:** 
***Наименьшая доля невозврата у категории бездетных. Наибольшая доля у семей с четыремя детьми. В целом, чем больше детей - тем больше доля невозврата, но не превышает 10 процентов. Уменьшение доли невозврата для семей с тремя детьми возможно связано с получением статуса многодетной семьи, то есть получением различных льгот от государства. Отсутствие невозвратов для семей с пятью детьми связано с небольшой представительской выборкой данных - в таблице представлено всего 9 семей и все они добропорядочные плательщики.
Общий вывод: наличие детей увеличивает долю невозврата на 2 процента по отношению к бездетным*** 


#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
# семейное положение - 'family_status'
# возврат кредита в срок - 'debt'
data_3 = data.groupby('family_status')['debt'].mean()*100
print(data_3)


family_status
Не женат / не замужем    9.763948
в разводе                7.064760
вдовец / вдова           6.624606
гражданский брак         9.313014
женат / замужем          7.560558
Name: debt, dtype: float64


**Вывод:** 
***Наибольший процент невозврата у людей с отсутствием опыта семейной жизни и у осторожных(гражданский брак). 
Наименьший - у одиноких. 
У семейных пар процент невозврата ниже, чем у не женатых, почти на 2 процента. 
В целом, семейное положение - бывшее или настоящее - снижает процент невозврата кредита в срок.*** 


#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
# уровень дохода - 'total_income'
# возврат кредита в срок - 'debt' 
# В описании проекта переменная debt — имел ли задолженность по возврату кредитов принимает значения:
# 1 - , истина, да, имел. Невозврат есть. 0 - ложь, нет, не имел. Невозвратов нет. Возврат в срок.

# У нас есть таблица total_income_category, в которой доходы распределены по категориям:
# 0–30000 — 'E';
# 30001–50000 — 'D';
# 50001–200000 — 'C';
# 200001–1000000 — 'B';
# 1000001 и выше — 'A'.

########### решение через groupby ###############
# посмотрим как распределяются доходы по категориям:
data_4 = data.groupby('total_income_category')['total_income'].count()
print(data_4)
# больше всего доходов в категории С - от 50 т.р. до 200 т.р.

# посчитаем среднюю долю невозврата кредита по каждой категории дохода
data_5 = data.groupby('total_income_category')['debt'].mean()*100
print(data_5)
# и рассчитаем средний уровень для всех категорий:
print(data.groupby('total_income_category')['debt'].mean().mean()*100)


total_income_category
A       25
B     5014
C    15921
D      349
E       22
Name: total_income, dtype: int64
total_income_category
A    8.000000
B    7.060231
C    8.498210
D    6.017192
E    9.090909
Name: debt, dtype: float64
7.733308466327596


**Вывод:** 
***Наибольшая доля невозвратов в категории доходов до 30 тысяч рублей - 'Е'. 
Наименьшая - в категории "бюджетников".  
При среднем доходе - от 50 до 200 т.р. - категория 'C' - доля невозвратов также остается высокой. 
Эта самая представительная категория.   
При высоких и очень высоких доходах процент невозврата кредитов остается на среднем уровне. 
В целом, чем ниже уровень дохода, тем выше процент невозврата. ***

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [25]:
# цель кредита - 'purpose' .
# Есть таблица 'purpose_category', в которой цели кредита распределены на 4 категории:
# 'операции с автомобилем',
# 'операции с недвижимостью',
# 'проведение свадьбы',
# 'получение образования'

# посчитаем среднюю долю невозврата кредита по каждой категории дохода
data_5 = data.groupby('purpose_category')['debt'].mean()*100
print(data_5)

purpose_category
операции с автомобилем      9.347978
операции с недвижимостью    7.255139
получение образования       9.252758
проведение свадьбы          7.911803
Name: debt, dtype: float64


**Вывод:** 
***Наименьший процент невозврата соответствует более серьезным целям: по операциям с недвижимостью и кредитам на свадьбу. 
Невозврат по автомобильным кредитам и на образование почти на два процента выше. 
В целом, семейное положение снижает процент невозврата кредита.***

**Приведем возможные причины появления пропусков в исходных данных.**

**Возможные причины появления пропусков связаны со сложностью заполнения дней занятости для пенсионеров и обучающихся. 
Также в этих категориях сложно указать доход ввиду его отсутствия.**


**Объясним, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.**

**Медианное значение менее чувствительно к аномальным значениям.**

### Шаг 4: общий вывод.

Целью исследования было изучение зависимости семейного положения и количества детей клиента на факт погашения кредита в срок.
Первоначальные данные были избавлены от пропусков, отсортированы и упорядочены. 
Мы проверили зависимости возврата кредита в срок от количества детей, семейного положения и уровня дохода, а также как разные цели влияют на возврат кредита в срок. 

<font color='darkblue'>
Для первой гипотезы "Есть ли зависимость между количеством детей и возвратом кредита в срок?"
мы установили, что наличие детей увеличивает долю невозврата на 2 процента по отношению к бездетным.
Для семей с тремя детьми и более необходимо увеличить количество данных. 
 
Для второй гипотезы "Есть ли зависимость между семейным положением и возвратом кредита в срок?" мы установили, что 
наибольший процент невозврата у людей с отсутствием опыта семейной жизни и у осторожных(гражданский брак). 
Наименьший - у одиноких. В этом исследовании мы установили, что семейное пложение - бывшее или настоящее - снижает процент невозврата кредита в срок. 

Для третьего исследования "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?" мы установили, что
гипотеза частично подтвердилась: чем ниже уровень дохода, тем выше процент невозврата. При высоких и очень высоких доходах процент невозврата кредитов остается на среднем уровне. Однако,в этих категориях очень мало данных для анализа. 
Также следует провести дополнительный анализ данных внутри самой представительной категории с доходами от 50 до 200 тысяч рублей.  

Для четвертого исследования "Как разные цели кредита влияют на его возврат в срок?" мы установили, что кредиты на автомобиль и образование имеют задолженность чаще, чем на свадьбу и недвижимость. Новое семейное положение и покупка недвижимости являются более обдуманными решениями, поэтому процент невозврата кредита в этих категориях снижен.
    
Возможно, следует провести исследование зависимости возврата кредитов от статуса заемщика: наемный работник, бизнесмен, иждивенец или пенсионер. 

Портрет ответственного заещика выглядит следующим образом: не имеет детей, находится\был в официальном браке, уровень дохода менее 50 тысяч рублей, целью кредита является недвижимость. Наименее ответственный заемщик имеет одного-двух детей, не женат(не замужем) или находится в гражданском браке, уровень дохода более 50 тысяч рублей, и берет кредит на свадьбу или обучение. 
    
    
</font>
